In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

# 1) Loading Adult Income Dataset (Census Income) dataset

In [2]:
# Link of the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

cols = [
    "age", "workclass", "fnlwgt", "education", "education_num",
    "marital_status", "occupation", "relationship", "race", "sex",
    "capital_gain", "capital_loss", "hours_per_week", "native_country", "income"
]

# Loading the dataset from link into pandas dataframe
adult = pd.read_csv(url, header = None, names = cols, na_values = " ?", skipinitialspace = True)

print(adult.shape)
adult.head()

(32561, 15)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
adult['income'] = label_encoder.fit_transform(adult['income'])

adult.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


# 2) Creating DataSet class
 ---------------------------------------------
 Note on `transform=None` in Dataset class:
 - `transform=None` → optional.
 - If a transform function is passed when creating the dataset, it is applied in __getitem__
   every time a sample is accessed.
 - If nothing is passed, the dataset just returns the raw/unprocessed data.
 - Acts like a “hook” for on-the-fly preprocessing without forcing it.
- This is the same pattern used in torchvision datasets like MNIST or CIFAR10.
 ---------------------------------------------


In [4]:
class MyDataset(Dataset):

  def __init__(self, data, label_col, transform = None):

    # Loading features and label from dataset
    self.x = data.drop(label_col, axis = 1)
    self.y = data[label_col]

    # Defining transformation, if a transformation function is passed by user while making the object of this class
    self.transform = transform


  def __len__(self):

    return len(self.x)    # self.x (or self.y) returns total number of samples

  def __getitem__(self, ind):

    # Get feature and label of the particular index
    x = self.x.iloc[ind]
    y = self.y.iloc[ind]

    if self.transform is not None:
      x = self.transform(x)                          # We get x as tensor after applying the transformation

    y = torch.tensor(y, dtype = torch.float32)       # Converting y to tensor too

    return x, y

# 3) Creating a transformation function

---

### 3.1) Encoding the Categorical Columns

We create a dictionary called `encoder` with the following structure:

- **Key** → Column name (e.g., `"workclass"`, `"education"`, etc.)  
- **Value** → Another dictionary that maps each category in that column to its corresponding label-encoded integer.

#### Example:
```
encoder = {
    "workclass": {
        "State-gov": 0,
        "Private": 1,
        "Self-emp": 2
    },
    "education": {
        "Bachelors": 0,
        "Masters": 1,
        "PhD": 2
    }
}
```

In [5]:
# Categorical  columns in the dataset
categorical_cols = ["workclass", "education", "marital_status", "occupation",
                    "relationship", "race", "sex", "native_country"]


# Encoding categorical values
encoder = {}                            # Dictionary storing mapping per column

for col in categorical_cols:

    unique_val = adult[col].unique()  # get all unique categories for that column
    mapping = {}                      # Dictionary for storing categories for 1 column

    label = 0
    for val in unique_val:
        mapping[val] = label
        label += 1

    encoder[col] = mapping            # store mapping for this column


for col in categorical_cols:          # Checking the encoder
    print(col, encoder[col])

print(encoder["workclass"]["Local-gov"], encoder['sex']['Female'])

workclass {'State-gov': 0, 'Self-emp-not-inc': 1, 'Private': 2, 'Federal-gov': 3, 'Local-gov': 4, '?': 5, 'Self-emp-inc': 6, 'Without-pay': 7, 'Never-worked': 8}
education {'Bachelors': 0, 'HS-grad': 1, '11th': 2, 'Masters': 3, '9th': 4, 'Some-college': 5, 'Assoc-acdm': 6, 'Assoc-voc': 7, '7th-8th': 8, 'Doctorate': 9, 'Prof-school': 10, '5th-6th': 11, '10th': 12, '1st-4th': 13, 'Preschool': 14, '12th': 15}
marital_status {'Never-married': 0, 'Married-civ-spouse': 1, 'Divorced': 2, 'Married-spouse-absent': 3, 'Separated': 4, 'Married-AF-spouse': 5, 'Widowed': 6}
occupation {'Adm-clerical': 0, 'Exec-managerial': 1, 'Handlers-cleaners': 2, 'Prof-specialty': 3, 'Other-service': 4, 'Sales': 5, 'Craft-repair': 6, 'Transport-moving': 7, 'Farming-fishing': 8, 'Machine-op-inspct': 9, 'Tech-support': 10, '?': 11, 'Protective-serv': 12, 'Armed-Forces': 13, 'Priv-house-serv': 14}
relationship {'Not-in-family': 0, 'Husband': 1, 'Wife': 2, 'Own-child': 3, 'Unmarried': 4, 'Other-relative': 5}
race {'

### 3.2) Scaling numerical class

We create a dictionary called `stats` with the following structure:

- **Key** → Column name (e.g., `"age"`, `"fnlwgt"`, `"hours_per_week"`, etc.)  
- **Value** → A tuple `(mean, std)` representing the mean and standard deviation of that column, calculated from the training dataset.

---

In [6]:
# Numerical columns in the dataset
numerical_cols = ["age", "fnlwgt", "education_num", "capital_gain",
                  "capital_loss", "hours_per_week"]

# Calculating mean and standard deviation for each row
stats = {}

for col in numerical_cols:
  mean = adult[col].mean()        # Calculating mean for this column
  std = adult[col].std()          # Calculating standard deviation for this column

  stats[col] = (mean, std)        # Storing mean, and std as a tuple

print(stats)

{'age': (np.float64(38.58164675532078), 13.640432553581146), 'fnlwgt': (np.float64(189778.36651208502), 105549.97769702233), 'education_num': (np.float64(10.0806793403151), 2.5727203320673406), 'capital_gain': (np.float64(1077.6488437087312), 7385.292084839299), 'capital_loss': (np.float64(87.303829734959), 402.960218649059), 'hours_per_week': (np.float64(40.437455852092995), 12.34742868173081)}


### 3.3) Writing the transformation function

In [7]:
def transformation(row):

  new_vals = {}                                  # Storing transformed values of the sample(row) in this dictonary

  for col, val in row.items():                   # .items() → (column_name, value)

    if col in categorical_cols:
      new_vals[col] = encoder[col][val]          # Replacing categorical value with encoded integer

    elif col in numerical_cols:
      mean, std = stats[col]
      new_vals[col] = (val - mean)/std           # Scaling numerical value

    else:                                        # Untouched columns if any- doing no transformation to those columns
      new_vals[col] = val


  # Convert to PyTorch tensor
  return torch.tensor(list(new_vals.values()), dtype=torch.float32)

### 3.4) Checking the transformation function is working properly

In [8]:
# Giving a row from the data, as it was given in Dataset class
row1 = adult.iloc[0]
print(type(row1))
row1

# Applying the transformations
row_new = transformation(row1)

print(type(row_new))
row_new

<class 'pandas.core.series.Series'>
<class 'torch.Tensor'>


tensor([ 0.0307,  0.0000, -1.0636,  0.0000,  1.1347,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.1485, -0.2167, -0.0354,  0.0000,  0.0000])

# 4) Splting the dataset into training and testing data

In [9]:
train_df, test_df = train_test_split(adult, test_size = 0.2, random_state = 42)

# 5) Creating Dataset & DataLoader Object

In [10]:
# Dataset object
train_data = MyDataset(train_df, "income", transform = transformation)
test_data = MyDataset(test_df, "income", transform = transformation)

# DataLoader object
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 6) Building ANN architecture using NN module

In [11]:
class AdultANN(nn.Module):

  def __init__(self, num_features):
    super(AdultANN, self).__init__()

    self.model = nn.Sequential(
        nn.Linear(num_features, 15),     # .Linear(number of input to this layer, number of neurons in this layer)
        nn.ReLU(),

        nn.Linear(15, 64),
        nn.ReLU(),

        nn.Linear(64, 1),                 # 1 neuron for binary classification
        nn.Sigmoid()                      # output layer activation function
    )

  def forward(self, x):                   # shape of x: [batch_size, num_features]

    y_pred = self.model(x)
    y_pred = y_pred.view(-1)              # [batch_size,1] → [batch_size] (matches BCELoss target shape).
    return y_pred

  def loss_function(self, y_pred, y):

    loss_fun = nn.BCELoss()
    loss_val = loss_fun(y_pred, y.float())
    return loss_val

# 7) Training the model on the dataset

In [12]:
# Initializing the model (Creating the model)
model = AdultANN(14)                              # num_features = 14 (number of features, or number of input columns)
print(model)

# Creating Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

AdultANN(
  (model): Sequential(
    (0): Linear(in_features=14, out_features=15, bias=True)
    (1): ReLU()
    (2): Linear(in_features=15, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [13]:
# Training the model
for epoch in range(25):

  for batch_features, batch_label in train_loader:

    # Forward pass
    y_pred = model(batch_features)

    # Loss calculation
    loss = model.loss_function(y_pred, batch_label)

    # Backward pass
    loss.backward()

    # Update the parameters
    optimizer.step()

    # Reset the gradients
    optimizer.zero_grad()

  print(f"Epoch {epoch+1}: Loss: {loss.item()}")

Epoch 1: Loss: 0.38213542103767395
Epoch 2: Loss: 0.36866921186447144
Epoch 3: Loss: 0.2151917666196823
Epoch 4: Loss: 0.25989460945129395
Epoch 5: Loss: 0.36349499225616455
Epoch 6: Loss: 0.4255756735801697
Epoch 7: Loss: 0.37151041626930237
Epoch 8: Loss: 0.21332037448883057
Epoch 9: Loss: 0.176153302192688
Epoch 10: Loss: 0.26470911502838135
Epoch 11: Loss: 0.39772263169288635
Epoch 12: Loss: 0.1924487054347992
Epoch 13: Loss: 0.21198530495166779
Epoch 14: Loss: 0.33305996656417847
Epoch 15: Loss: 0.1624687761068344
Epoch 16: Loss: 0.46886906027793884
Epoch 17: Loss: 0.27117711305618286
Epoch 18: Loss: 0.253571093082428
Epoch 19: Loss: 0.47761303186416626
Epoch 20: Loss: 0.3665573298931122
Epoch 21: Loss: 0.3021260201931
Epoch 22: Loss: 0.4579237699508667
Epoch 23: Loss: 0.3332894444465637
Epoch 24: Loss: 0.2643158733844757
Epoch 25: Loss: 0.08969477564096451


# 8) Evaluating the model

In [14]:
# Seting model to eval mode
model.eval()

AdultANN(
  (model): Sequential(
    (0): Linear(in_features=14, out_features=15, bias=True)
    (1): ReLU()
    (2): Linear(in_features=15, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [15]:
# Making prediction on test data

total_samples = 0
correct_pred = 0

with torch.no_grad():

  for batch_features, batch_label in test_loader:

    # Forward pass
    y_test_pred = model(batch_features)

    # Converting probability values to predicted class
    y_pred = (y_test_pred > 0.8).float()

    # Check if correctly predicted
    correct_pred += (y_pred == batch_label).sum().item()

    # Total samples
    total_samples += batch_label.shape[0]


# Calculating accuracy
acc = correct_pred/total_samples
print('Accuracy: ', acc*100)

Accuracy:  80.94580070627974


In [16]:
# Evaluating to training data

total_samples = 0
correct_pred = 0

with torch.no_grad():

  for batch_features, batch_label in train_loader:

    # Forward pass
    y_train_pred = model(batch_features)

    # Converting probability values to predicted class
    y_pred = (y_train_pred > 0.8).float()

    # Check if correctly predicted
    correct_pred += (y_pred == batch_label).sum().item()

    # Total samples
    total_samples += batch_label.shape[0]


# Calculating accuracy
acc = correct_pred/total_samples
print('Accuracy: ', acc*100)

Accuracy:  80.60887592137593


#### Why Calculate Training Accuracy After Testing Accuracy?

- **Testing Accuracy** → shows how well the model generalizes to unseen data.  
- **Training Accuracy** → shows how well the model fits the data it was trained on.  

Comparing both helps diagnose model behavior:  

| Training Accuracy | Testing Accuracy | Interpretation |
|-------------------|------------------|----------------|
| High              | High             | ✅ Good fit (generalizes well) |
| High              | Low              | ⚠️ Overfitting |
| Low               | Low              | ⚠️ Underfitting |
| Medium            | Medium           | 🤔 Needs more training/features |

> Calculating both in the **same manner** ensures results are consistent and directly comparable. Compare both → helps detect overfitting or underfitting
